# Centralizing the Data Vizualizations from the Paper

In [ ]:
#path configuration
to_data_parent = "../Data/abby_ceo" #path to the parent folder, should contain program subfolders
to_data_folder = "../Data" #data folder, where composite data was written from the TSDC_data file

In [ ]:
# dependencies
from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import linear_model

sns.set_style("whitegrid")
sns.set()
%matplotlib inline

params = {'legend.fontsize': 'small',
          'figure.figsize': (10, 8),
         'axes.labelsize': 'small',
         'axes.titlesize':'small',
         'xtick.labelsize':'small',
         'ytick.labelsize':'small'}
plt.rcParams.update(params)


import importlib

## full data - for labeling rates

In [ ]:
#gather all of the trips into one dataframe
#loop over
programs = ['4c', 'cc', 'fc', 'pc', 'sc', 'vail_22']
datasets = []

for program in programs:
    print('starting with ', program)
    
    #create dataset with surveys and trips
    trips = pd.read_csv(to_data_parent + '/' + program + '/analysis_confirmed_trip.csv')
    print(len(trips), 'trips')
    print(trips.perno.nunique(), 'people')

    #prepare trip ids for merging
    trips['user_id_socio'] = trips.perno.astype(str)
    trips['user_id_socio'] = trips['user_id_socio'].str.strip() #remove leading or trailing whitespace!!
    trips.user_id_socio = [i.replace('-','') for i in trips.user_id_socio] # remove all dashes from strings
    
    trips['program'] = program.split('_')[0]
    
    #add to list of datasets
    datasets.append(trips)

participant_ct_df = pd.concat(datasets)

#just labeled data
labeled_df = participant_ct_df[participant_ct_df.data_user_input_mode_confirm.notna() | 
                                       participant_ct_df.data_user_input_purpose_confirm.notna() |
                                       participant_ct_df.data_user_input_replaced_mode.notna()]

In [ ]:
all_user_trips = participant_ct_df.groupby(['user_id_socio'], as_index=False).count()[['user_id_socio','data_distance']]
print(len(all_user_trips))

labeled_user_trips = labeled_df.groupby(['user_id_socio'], as_index=False).count()[['user_id_socio','data_distance']]
print(len(labeled_user_trips))

plot_data = all_user_trips.merge(labeled_user_trips, how='right', on='user_id_socio').fillna(0)
plot_data.head()

### figure 2

In [ ]:
plot_data['proportion'] = plot_data['data_distance_y'] / plot_data['data_distance_x']
data_order = plot_data.sort_values('proportion', ascending=True).user_id_socio
print(len(plot_data))

plot_title='Distribution of User Response Rates'
ylab='Proportion of Trips Labeled'
file_name='CanBikeCO_report_user_participation%s'
fig, ax = plt.subplots(figsize=(10,4))
sns.barplot(data=plot_data, x='user_id_socio', y='proportion', order=data_order, color='blue').set(title=plot_title,xlabel='Individual Users (176)',ylabel=ylab,xticklabels=[])
plt.subplots_adjust(bottom=0.25)
ax.figure.savefig(file_name+".jpg", bbox_inches='tight')

In [ ]:
labeled_user_trips.columns

In [ ]:
#drop infected data
all_data = participant_ct_df.copy()

all_data['data_start_local_dt_month'] = all_data['data_start_local_dt_month'].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int).dropna()
all_data = all_data[all_data.data_start_local_dt_month >= 1]
all_data = all_data[all_data.data_start_local_dt_month <= 12]

all_data['data_start_local_dt_year'] = all_data['data_start_local_dt_year'].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int).dropna()
all_data = all_data[all_data.data_start_local_dt_year >= 2019]
all_data = all_data[all_data.data_start_local_dt_year <= 2023]

labeled = labeled_df.copy()
labeled['data_start_local_dt_month'] = labeled['data_start_local_dt_month'].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int).dropna()
labeled = labeled[labeled.data_start_local_dt_month >= 1]
labeled = labeled[labeled.data_start_local_dt_month <= 12]

labeled['data_start_local_dt_year'] = labeled['data_start_local_dt_year'].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int).dropna()
labeled = labeled[labeled.data_start_local_dt_year >= 2019]
labeled = labeled[labeled.data_start_local_dt_year <= 2023]

In [ ]:
#group the total data by day
all_data = (all_data.groupby(['data_start_local_dt_month', 'data_start_local_dt_year']).size() 
   .reset_index(name='count'))

all_data = all_data.sort_values(['data_start_local_dt_year', 'data_start_local_dt_month'])

all_data = all_data.astype({'data_start_local_dt_month': 'str'})
all_data = all_data.astype({'data_start_local_dt_year': 'str'})
all_data['Month'] = all_data[['data_start_local_dt_year', 'data_start_local_dt_month']].agg('-'.join, axis=1)

#group the labeled data by day#group the total data by day
labeled = (labeled.groupby(['data_start_local_dt_month', 'data_start_local_dt_year']).size() 
   .reset_index(name='count'))

labeled = labeled.sort_values(['data_start_local_dt_year', 'data_start_local_dt_month'])

labeled = labeled.astype({'data_start_local_dt_month': 'str'})
labeled = labeled.astype({'data_start_local_dt_year': 'str'})
labeled['Month'] = labeled[['data_start_local_dt_year', 'data_start_local_dt_month']].agg('-'.join, axis=1)

#merge them
plot_data = all_data.merge(labeled, how='left', on='Month').fillna(0)

#calc the proportion
plot_data['proportion'] = plot_data['count_y'] / plot_data['count_x']

#drop data before 6/2022 (when the full pilot started)
plot_data = plot_data.iloc[5:]

### figure 3

In [ ]:
#graph it - bar chart

plot_title='Response Rates Over Time'
ylab='Proportion of Trips Labeled'
file_name='CanBikeCO_report_ts_labels'
fig, ax = plt.subplots(figsize=(8,4))
sns.barplot(data=plot_data, x='Month', y='proportion', color='blue').set(title=plot_title,xlabel='Month',ylabel=ylab)
plt.xticks(rotation=35, ha='right', fontsize=10)
plt.subplots_adjust(bottom=0.25)
ax.figure.savefig(file_name+".jpeg", bbox_inches='tight')

## Loading from CSV way

In [ ]:
# loading the data
cleaned_data = pd.read_csv(to_data_folder + "/tsdc_filtered_merged_trips.csv")

In [ ]:
# Summary statistics table
print(len(pd.unique(cleaned_data.user_id)))
stat_data = cleaned_data[['distance','duration']]
stat_data.describe()

## general demographics

figure #10

In [ ]:
# Age, Income, Gender
plot_data = cleaned_data.copy()
plot_data = plot_data.groupby(['user_id']).nth(0)[['AGE','GENDER','VEH_num','HHINC']].dropna()
plot_data = plot_data[plot_data['GENDER'].isin(['Man','Woman'])]
plot_data = plot_data.groupby(['user_id'], as_index=False).nth(0)

plot_title='Participant Demographics'
ylab='Count'
file_name='CanBikeCO_report_demog'
fig, axs = plt.subplots(2,2,figsize=(10,6))
sns.histplot(data=plot_data, x='GENDER', ax=axs[0,0], color='purple', stat='probability').set(xlabel='Sex',ylabel='proportion')
sns.histplot(data=plot_data, x='AGE', ax=axs[0,1], color='red', stat='probability').set(xlabel='Age',ylabel='proportion')
sns.histplot(data=plot_data, x='VEH_num', ax=axs[1,0], color='blue', stat='probability').set(xlabel='Household Vehicles',ylabel='proportion')
sns.histplot(data=pd.DataFrame(plot_data['HHINC'].dropna()), x='HHINC', ax=axs[1,1], color='green', stat='probability').set(xlabel='Household Income',ylabel='proportion')
plt.xticks(rotation=35, ha='right')
plt.tight_layout()

fig.savefig(file_name+".png", bbox_inches='tight')

## Modes in Mini vs Full Pilot

Figure 6 in the paper

In [ ]:
# processing mini data
mini_pilot_trips = pd.read_csv(to_data_folder + "/trip_program.csv")
mini_pilot_trips = mini_pilot_trips[mini_pilot_trips.program == 'prepilot']
print(len(mini_pilot_trips))
MINI_PILOT_DF = mini_pilot_trips.copy() #saving a copy for later

# Combine variable categories
mini_pilot_trips = mini_pilot_trips.replace('Gas Car, drove alone', 'Car')
mini_pilot_trips = mini_pilot_trips.replace('Gas Car, with others', 'Shared Car')
mini_pilot_trips = mini_pilot_trips.replace('Bikeshare', 'Shared Micromobility')
mini_pilot_trips = mini_pilot_trips.replace('Scooter share', 'Shared Micromobility')
mini_pilot_trips = mini_pilot_trips.replace('Regular Bike', 'Personal Micromobility')
mini_pilot_trips = mini_pilot_trips.replace('Skate board', 'Personal Micromobility')
mini_pilot_trips = mini_pilot_trips.replace('Train', 'Transit')
mini_pilot_trips = mini_pilot_trips.replace('Free Shuttle', 'Transit')
mini_pilot_trips = mini_pilot_trips.replace('Bus', 'Transit')
mini_pilot_trips = mini_pilot_trips.replace('Walk', 'Walk')
mini_pilot_trips = mini_pilot_trips.replace('Taxi/Uber/Lyft', 'Ridehail')
mini_pilot_trips = mini_pilot_trips.replace('Pilot ebike', 'E-Bike')

#filter out 'not a trip' trips
mini_pilot_trips = mini_pilot_trips[~mini_pilot_trips['Mode_confirm'].isin(['Not a Trip'])]
mini_pilot_trips = mini_pilot_trips[~mini_pilot_trips['Replaced_mode'].isin(['Not a Trip'])]
mini_pilot_trips = mini_pilot_trips[~mini_pilot_trips['Trip_purpose'].isin(['not_a_trip'])]

print(len(mini_pilot_trips))

In [ ]:
#processes dataframe for stacked bars - commute vs all trips
#input: dataframe of all trips, flag to insert missing "Other" row
#output: formatted df of work trips and all trips with counts and percentages for modes
def format_for_mode_bars(trip_df, dataset, insert_other=False):
  all_trips_df = trip_df.copy()

  all_trips_df.loc[all_trips_df['Mode_confirm']=='Personal Micromobility', 'Mode_confirm'] = 'Other'
  all_trips_df.loc[all_trips_df['Mode_confirm']=='Shared Micromobility', 'Mode_confirm'] = 'Other'  

  all_trips = all_trips_df.groupby(['Mode_confirm'], as_index=False).count()[['Mode_confirm','distance_miles']]
  all_trips['proportion'] = all_trips['distance_miles'] / np.sum(all_trips.distance_miles)
  all_trips['trip_type'] = 'All Trips'

  work_trips = all_trips_df[all_trips_df['Trip_purpose']=='Work'].copy()
  work_trips = work_trips.groupby(['Mode_confirm'], as_index=False).count()[['Mode_confirm','distance_miles']]
  work_trips['proportion'] = work_trips['distance_miles'] / np.sum(work_trips.distance_miles)
  work_trips['trip_type'] = 'Work Trips'

  if insert_other:
    work_trips.loc[1.5] = 'Other', 0, 0, 'Work Trips'
    work_trips = work_trips.sort_index().reset_index(drop=True)

  formatted_df = pd.concat([all_trips,work_trips])
  formatted_df['Dataset'] = dataset
  formatted_df.columns = ['Mode','Count','Proportion','Trip Type', "Dataset"]

  return formatted_df

In [ ]:
#prepare mini and full trip dfs for plot
mini_trips_cleaned = format_for_mode_bars(mini_pilot_trips, 'Minipilot', True)
full_trips_cleaned = format_for_mode_bars(cleaned_data, 'Long Term')

plot_data = pd.concat([full_trips_cleaned, mini_trips_cleaned])
plot_data

In [ ]:
#input: dataframe formatted for plotting, dimension (Mode or Purpose)
def make_mini_vs_full(plot_data, dimension):
    width = 0.8
    fig, ax = plt.subplots(2,1, figsize=(20,10))
    plt.rcParams.update({'font.size': 30}) 
    running_total_mini = [0,0]
    running_total_long = [0,0]
    fig_data_mini = plot_data[plot_data['Dataset']=='Minipilot']
    fig_data_long = plot_data[plot_data['Dataset']=='Long Term']

    for mode in pd.unique(fig_data_mini[dimension]):
        mini = fig_data_mini[fig_data_mini[dimension]==mode]
        long = fig_data_long[fig_data_long[dimension]==mode]

        labels = mini['Trip Type']
        vals = mini['Proportion']*100
        bar_labels = mini['Count']
        vals_str = [f'{y:.1f} %\n({x:,})' if y>5 else '' for x, y in zip(bar_labels, vals)]
        bar = ax[0].barh(labels, vals, width, left=running_total_mini, label=mode)
        ax[0].bar_label(bar, label_type='center', labels=vals_str, rotation=90, fontsize=22)
        running_total_mini[0] = running_total_mini[0]+vals.iloc[0]
        running_total_mini[1] = running_total_mini[1]+vals.iloc[1]

        labels = long['Trip Type']
        vals = long['Proportion']*100
        bar_labels = long['Count']
        vals_str = [f'{y:.1f} %\n({x:,})' if y>5 else '' for x, y in zip(bar_labels, vals)]
        bar = ax[1].barh(labels, vals, width, left=running_total_long, label=mode)
        ax[1].bar_label(bar, label_type='center', labels=vals_str, rotation=90, fontsize=22)
        running_total_long[0] = running_total_long[0]+vals.iloc[0]
        running_total_long[1] = running_total_long[1]+vals.iloc[1]

    ax[0].set_title('Minipilot', fontsize=25)
    ax[1].set_title('All Programs', fontsize=25)
    ax[0].legend(bbox_to_anchor=(1,1), fancybox=True, shadow=True, fontsize=25)
    plt.subplots_adjust(bottom=0.20)
    fig.tight_layout()
    plt.show()

In [ ]:
# make and save the chart
make_mini_vs_full(plot_data, 'Mode')

## Mode Share by Program

figure 7 in the paper

In [ ]:
# process the data for the clustered chart
cleaned_data.program.unique()

In [ ]:
def format_mode_by_program(program_data, program_list, work = False):
    subset_plot_data = []
    for program in program_list:
        program_data = mode_data[mode_data.program == program]

        if work:
            program_data = program_data[program_data['Trip_purpose']=='Work'].copy()
            
        formatted_df = program_data.groupby(['Mode_confirm'], as_index=False).count()[['Mode_confirm','distance_miles']]
        formatted_df[program] = (formatted_df['distance_miles'] / np.sum(formatted_df.distance_miles)) * 100
        formatted_df = formatted_df.set_index('Mode_confirm')
        formatted_df = formatted_df.drop(columns = ['distance_miles'])
        subset_plot_data.append(formatted_df)


    formatted_trips = pd.concat(subset_plot_data, axis = 1)
    formatted_trips = formatted_trips.transpose()
    formatted_trips['program'] = formatted_trips.index
    formatted_trips = formatted_trips.replace({'4c': 'Four Corners\n(Durango)', 
                                   'cc': 'Comunity Cycles\n(Boulder)',
                                   'sc': 'Smart Commute\n(Denver North)',
                                   'pc':'Pueblo',
                                   'vail':'Vail',
                                   'fc':'Fort Collins'})
    formatted_trips = formatted_trips.set_index('program')

    return formatted_trips

In [ ]:
mode_data = cleaned_data.copy()

#clean up the modes
mode_data.loc[mode_data['Mode_confirm']=='Personal Micromobility', 'Mode_confirm'] = 'Other'
mode_data.loc[mode_data['Mode_confirm']=='Shared Micromobility', 'Mode_confirm'] = 'Other'

program_list = ['4c', 'cc', 'fc', 'pc', 'sc', 'vail']

work_trips = format_mode_by_program(mode_data, program_list, True)
all_trips = format_mode_by_program(mode_data, program_list, False)

In [ ]:
##COLUMN ORDERS MUST MATCH OR CHART MISREPRESENTS DATA
all_trips

In [ ]:
work_trips['Ridehail'] = work_trips['Ridehail'].fillna(0)
work_trips['Transit'] = work_trips['Transit'].fillna(0)
work_trips = work_trips[['Car', 'E-bike', 'Other', 'Ridehail', 'Shared Car', 'Transit', 'Walk']]

work_trips

### make and save the clustered chart

In [ ]:
#from https://stackoverflow.com/questions/22787209/how-to-have-clusters-of-stacked-bars
import pandas as pd
import matplotlib.cm as cm
import numpy as np
import matplotlib.pyplot as plt

def plot_clustered_stacked(dfall, labels=None, title="multiple stacked bar plot",  H="/", **kwargs):
    """Given a list of dataframes, with identical columns and index, create a clustered stacked bar plot. 
labels is a list of the names of the dataframe, used for the legend
title is a string for the title of the plot
H is the hatch used for identification of the different dataframe"""

    n_df = len(dfall)
    n_col = len(dfall[0].columns) 
    n_ind = len(dfall[0].index)
    axe = plt.subplot(111)

    for df in dfall : # for each data frame
        axe = df.plot(kind="bar",
                      stacked=True,
                      ax=axe,
                      legend=False,
                      grid=False,
                      **kwargs)  # make bar plots

    h,l = axe.get_legend_handles_labels() # get the handles we want to modify
    for i in range(0, n_df * n_col, n_col): # len(h) = n_col * n_df
        for j, pa in enumerate(h[i:i+n_col]):
            for rect in pa.patches: # for each index
                rect.set_x(rect.get_x() + 1 / float(n_df + 1) * i / float(n_col))
                rect.set_hatch(H * int(i / n_col)) #edited part     
                rect.set_width(1 / float(n_df + 1))

    axe.set_xticks((np.arange(0, 2 * n_ind, 2) + 1 / float(n_df + 1)) / 2.)
    axe.set_xticklabels(df.index, rotation = 0)
    axe.set_title(title, fontsize = 22)

    # Add invisible data to add another legend
    n=[]        
    for i in range(n_df):
        n.append(axe.bar(0, 0, color="gray", hatch=H * i))

    l1 = axe.legend(h[:n_col], l[:n_col], loc=[1.01, 0.5])
    if labels is not None:
        l2 = plt.legend(n, labels, loc=[1.01, 0.1]) 
    axe.add_artist(l1)
    return axe

In [ ]:
# Then, just call :
ax = plot_clustered_stacked([all_trips, work_trips],["All Trips", "Work Trips"], title = "Mode Share by Program")

for c in ax.containers:
    labels = [f'{round(v.get_height())}' if v.get_height() > 5 else '' for v in c]
    ax.bar_label(c, labels=labels, label_type='center')
    
ax.set_xticklabels(all_trips.index, rotation=45, ha='right', fontsize=14)

ax.set_xlabel('Program', fontsize = 18)
ax.set_ylabel('Proportion of Total Trip Count (%)', fontsize = 18)

plt.savefig("CanBikeCO_report_mode_share_overview.jpeg", bbox_inches='tight')

## Trip Purpose Mini vs Full

figure 8 in the paper

In [ ]:
def format_purpose_bars(trip_df, dataset):
    trips = trip_df.copy()
    trips = trips[~trips['Trip_purpose'].isin(['No travel'])]
    trips.loc[trips['Trip_purpose']=='Religious', 'Trip_purpose'] = 'Other'
    trips.loc[trips['Trip_purpose']=='School', 'Trip_purpose'] = 'Other'

    all_trips = trips.groupby(['Trip_purpose'], as_index=False).count()[['Trip_purpose','distance_miles']]
    all_trips['proportion'] = all_trips['distance_miles'] / np.sum(all_trips.distance_miles)
    all_trips['trip type'] = 'All Trips'

    ebike_trips = trips[trips['Mode_confirm']=='E-bike'].copy()
    ebike_trips = ebike_trips.groupby(['Trip_purpose'], as_index=False).count()[['Trip_purpose','distance_miles']]
    ebike_trips['proportion'] = ebike_trips['distance_miles'] / np.sum(ebike_trips.distance_miles)
    ebike_trips['trip type'] = 'E-Bike Trips'
    
    if dataset == "Long Term": #full dataset needs pickup/dropoff added
        all_trips.loc[len(all_trips.index)] = ['Pick-up/Drop off', 0, 0, 'All Trips']
        ebike_trips.loc[len(ebike_trips.index)] = ['Pick-up/Drop off', 0, 0, 'E-Bike Trips']

    formatted_trips = pd.concat([all_trips, ebike_trips])
    formatted_trips['Dataset'] = dataset
    formatted_trips.columns = ['Purpose','Count','Proportion','Trip Type', "Dataset"]

    return formatted_trips

In [ ]:
#arrange the data
full_purpose_data = cleaned_data.copy()
mini_purpose_data = MINI_PILOT_DF.copy()

mini_pilot_trips = format_purpose_bars(mini_purpose_data, "Minipilot")
plot_data = format_purpose_bars(full_purpose_data, "Long Term")

plot_data = pd.concat([plot_data, mini_pilot_trips])

In [ ]:
# create the chart
make_mini_vs_full(plot_data, 'Purpose')

## Show what purposes different programs used the e-bikes for

figure 9 in the paper

In [ ]:
full_purpose_data = cleaned_data.copy()
e_purpose = full_purpose_data[full_purpose_data.Mode_confirm == 'E-bike']

e_purpose.loc[e_purpose['Trip_purpose']=='Religious', 'Trip_purpose'] = 'Other'
e_purpose.loc[e_purpose['Trip_purpose']=='School', 'Trip_purpose'] = 'Other'

program_list = ['4c', 'cc', 'fc', 'pc', 'sc', 'vail']
all_plot_data = []
for program in program_list:
    program_data = e_purpose[mode_data.program == program]

    t1 = program_data.groupby(['Trip_purpose'], as_index=False).sum()[['Trip_purpose','distance_miles']]
    t1['distance_miles'].fillna(0, inplace=True)
    t1[program] = (t1['distance_miles'] / np.sum(t1.distance_miles)) * 100
    t1 = t1.set_index('Trip_purpose')
    t1 = t1.drop(columns = ['distance_miles'])
    all_plot_data.append(t1)
    
all_trips = pd.concat(all_plot_data, axis = 1)

all_trips = all_trips.transpose()

all_trips['program'] = all_trips.index
all_trips = all_trips.replace({'4c': 'Four Corners\n(Durango)', 
                               'cc': 'Comunity Cycles\n(Boulder)',
                               'sc': 'Smart Commute\n(Denver North)',
                               'pc':'Pueblo',
                               'vail':'Vail',
                               'fc':'Fort Collins'})
all_trips = all_trips.set_index('program')
# all_trips = all_trips.drop(columns=['Trip_purpose'])

all_trips

In [ ]:
#https://stackoverflow.com/questions/41296313/stacked-bar-chart-with-centered-labels
ax = all_trips.plot.bar(stacked=True, 
                        title="E-bike Purpose Share by Program", 
                        ylabel="Proportion E-bike Mileage (%)",
                        xlabel = "Program",
                        rot = 45)

for c in ax.containers:
    labels = [f'{round(v.get_height())}' if v.get_height() > 5.5 else '' for v in c]
    ax.bar_label(c, labels=labels, label_type='center')

ax.set_xticklabels(all_trips.index, rotation=45, ha='right')
ax.legend(bbox_to_anchor=(1,1), fancybox=True, shadow=True, fontsize=20)

plt.savefig("CanBikeCO_report_ebike_programs_purp_share.jpeg", bbox_inches='tight')


## Mode share by age

figure 11 in the paper

In [ ]:
mode_data = cleaned_data.copy() #complete set of cleaned data

age_counts = mode_data.groupby(['AGE'], as_index=False).count()[['AGE', 'user_id']]

bins = [18, 30, 42, 54, 66]
mode_data['age_bin'] = pd.cut(mode_data['AGE'], bins)

age_modes = mode_data.groupby(['age_bin', 'Mode_confirm'], as_index=False).count()[['age_bin', 'Mode_confirm','distance_miles']]
age_modes['proportion'] = age_modes['distance_miles'] / np.sum(age_modes.distance_miles)

list_age_modes = []
for age_bin in age_modes.age_bin.unique():
    age_data = mode_data[mode_data['age_bin'] == age_bin]

    age_bin_df = age_data.groupby(['Mode_confirm'], as_index=False).count()[['Mode_confirm','distance_miles']]
    age_bin_df['distance_miles'].fillna(0, inplace=True)
    age_bin_df[age_bin] = (age_bin_df['distance_miles'] / np.sum(age_bin_df.distance_miles)) * 100
    age_bin_df = age_bin_df.set_index('Mode_confirm')
    age_bin_df = age_bin_df.drop(columns = ['distance_miles'])
    list_age_modes.append(age_bin_df)
        
age_modes = pd.concat(list_age_modes, axis = 1)
age_modes = age_modes.transpose()
age_modes = age_modes.fillna(0)

age_modes

In [ ]:
#https://stackoverflow.com/questions/41296313/stacked-bar-chart-with-centered-labels
ax = age_modes.plot.bar(stacked=True, 
                        title="Mode Share by Age", 
                        ylabel="Proportion of Total Trip Count (%)",
                        xlabel = "Participant Age",
                        rot = 0)

for c in ax.containers:
    labels = [f'{round(v.get_height())}' if v.get_height() > 5.5 else '' for v in c]
    ax.bar_label(c, labels=labels, label_type='center')

# ax.set_xticklabels(age_modes.index, rotation=45, ha='right')
ax.legend(bbox_to_anchor=(1,1), fancybox=True, shadow=True, fontsize=20)

ax.set_xlabel("Participant Age", fontsize = 18)
ax.set_ylabel("Proportion of Total Trip Count (%)", fontsize = 18)
ax.set_title("Mode Share by Age", fontsize = 22)

plt.savefig("CanBikeCO_report_age_mode_share.jpeg", bbox_inches='tight')

## e-bike distances by program

figure 5 in the paper

In [ ]:
# Distribution of distances by program
plot_data = cleaned_data.copy()
plot_data = plot_data[plot_data['Mode_confirm']=='E-bike']
plot_data['Program'] = plot_data['program'].replace(['4c','cc','fc','pc','sc','vail'],['Four Corners','Community Cycles\n(Boulder)','Fort Collins','Pueblo','Smart Commute\n(Denver North)','Vail'])

plot_title = 'Distribution of E-Bike Trip Distances by Program'
ylab = 'Distance (miles)'

fig, ax = plt.subplots(figsize=(10,8))
sns.boxplot(ax=ax, data=plot_data, x='Program', y='distance_miles', hue='Mode_confirm', showfliers=False).set(title=plot_title, xlabel='', ylabel=ylab)
plt.subplots_adjust(bottom=0.25)
plt.xticks(rotation=35, ha='right', fontsize=14)
plt.yticks(fontsize=14)
plt.legend([])

# Calculate number of obs per group & median to position labels
medians = plot_data.groupby(['Program'])['distance_miles'].median().values
nobs = plot_data['Program'].value_counts().values
nobs = [str(x) for x in nobs.tolist()]
nobs = ["n: " + i for i in nobs]
 
# Add it to the plot
pos = range(len(nobs))
for tick,label in zip(pos,ax.get_xticklabels()):
    ax.text(pos[tick],
            medians[tick] + 0.03,
            nobs[tick],
            horizontalalignment='center',
            size='12',
            color='w',
            weight='semibold')
 
plt.savefig("CanBikeCO_report_e-bike_miles_dist.jpeg", bbox_inches='tight')

## E-bike trips across occuptations

figures 13 and 25

In [ ]:
cleaned_data.columns

In [ ]:
data = cleaned_data.copy()
data['occupation_cat'] = data['which_best_describes_your_primary_job?'].replace(['Sales or service',
                                                                                 'Manufacturing, construction, maintenance, or farming',                                                                            
                                                                                 'Janitorial',
                                                                                 'Professional, managerial, or technical',
                                                                                 'Clerical or administrative support',
                                                                                 'Teacher',
                                                                                 'Medical',
                                                                                 'CNA',
                                                                                 'Restaurant manager',
                                                                                 'Co op laundry',
                                                                                 'Cook',
                                                                                 'Nurse',
                                                                                 'Dining Services',
                                                                                 'Security',
                                                                                 'Food service',
                                                                                 'Csu custodian',
                                                                                 'Residential Dining Services',
                                                                                 'education/early childhood',
                                                                                 'Inbound cs',
                                                                                 'Custodial Maintanace',
                                                                                 'Amazon',
                                                                                 'Custodian',
                                                                                 'Hockey rink',
                                                                                 'Pastry chef and line cook',                                                                                 
                                                                                 'Cooking',
                                                                                 'Education non-profit manager',
                                                                                 'Healthcare',
                                                                                 'Chef',
                                                                                 'Accounting Technician',
                                                                                 'Caregiver/ Qmap',
                                                                                 'Caregiver',
                                                                                 'Health care',
                                                                                 'Medical field'],
                                                                                ['Sales or Service',
                                                                                 'Manufacturing, Construction, Maintenance, or Farming',
                                                                                 'Custodial',
                                                                                 'Professional, Managerial, or Technical',
                                                                                 'Clerical or Administrative Support',
                                                                                 'Education',
                                                                                 'Medical/Healthcare',
                                                                                 'Medical/Healthcare',
                                                                                 'Professional, Managerial, or Technical',
                                                                                 'Sales or Service',
                                                                                 'Sales or Service',
                                                                                 'Medical/Healthcare',
                                                                                 'Sales or Service',
                                                                                 'Professional, Managerial, or Technical',
                                                                                 'Sales or Service',
                                                                                 'Custodial',
                                                                                 'Sales or Service',
                                                                                 'Education',
                                                                                 'Professional, Managerial, or Technical',
                                                                                 'Custodial',
                                                                                 'Sales or Service',
                                                                                 'Custodial',
                                                                                 'Sales or Service',
                                                                                 'Sales or Service',
                                                                                 'Sales or Service',
                                                                                 'Education',
                                                                                 'Medical/Healthcare',
                                                                                 'Sales or Service',
                                                                                 'Professional, Managerial, or Technical',
                                                                                 'Medical/Healthcare',
                                                                                 'Medical/Healthcare',
                                                                                 'Medical/Healthcare',
                                                                                 'Medical/Healthcare'])
data['occupation_cat'].unique()

In [ ]:
data['occupation_cat']= data['occupation_cat'].replace(['Food Service', 'Cooking ', 'Accounting Technician ','Education ',
                                                       'Csu custodian ','Custodial ','Maintenance ','Maintenance','Janitorial ',
                                                       'Amazon ', 'Custodial Maintanace ', 'Hockey rink '],
                                                      ['Sales or Service' , 'Sales or Service', 'Professional, Managerial, or Technical',
                                                      'Education', 'Custodial', 'Custodial', 'Custodial', 'Custodial', 'Custodial',
                                                      'Sales or Service','Custodial' ,'Sales or Service'])

data['occupation_cat'] = data['occupation_cat'].replace(['Manufacturing, Construction, Maintenance, or Farming', 'Professional, Managerial, or Technical', 'Clerical or Administrative Support'],
                                                        ['Manufacturing, Construction,\nMaintenance, or Farming', 'Professional, Managerial,\nor Technical', 'Clerical or\nAdministrative Support'])
data['occupation_cat'].unique()

In [ ]:
plot_data_1=data[data['occupation_cat'].notnull()]

In [ ]:
# proportion of trips by occupation
plot_data = plot_data_1.copy()

t1 = plot_data.groupby(['user_id','Mode_confirm'], as_index=False).count()[['user_id','Mode_confirm','distance_miles']]
t1['distance_miles'].fillna(0, inplace=True)
t2 = plot_data.groupby(['user_id'], as_index=False).count()[['user_id','distance_miles']]
plot_data = t1.merge(t2, on='user_id')
plot_data['proportion'] = plot_data['distance_miles_x'] / plot_data['distance_miles_y']
plot_data['proportion'].fillna(0, inplace=True)
t3 = data.copy().groupby(['occupation_cat','user_id'], as_index=False).nth(0)[['occupation_cat','user_id']]

plot_data = plot_data[plot_data['Mode_confirm']=='E-bike']
plot_data = plot_data.merge(t3, on='user_id')

plot_title = 'E-bike Use (Trips) by Occupation Categories'
ylab='Occupation Category'
fig, ax = plt.subplots(figsize=(8,6))
sns.barplot(data= plot_data, x='proportion' , y='occupation_cat', estimator=np.mean).set(title=plot_title,xlabel='Proportion of Total Trips',ylabel=ylab)
plt.xticks(rotation=35, ha='right')
plt.subplots_adjust(bottom=0.25)

plt.savefig("CanBikeCO_report_occ_ebike_trips.jpeg", bbox_inches='tight')

In [ ]:
data['induced']=np.where(data['Replaced_mode']=='No Travel', 'Induced', 'Non-induced')

In [ ]:
data['Program'] = data['program'].replace(['4c','cc','fc','pc','sc','vail'],['Four Corners (Durango)','Community Cycles (Boulder)','Fort Collins','Pueblo County','Smart Commute (Denver North)','Vail'])

In [ ]:
# proportion of induced trips by occupation
plot_data = data.copy()
plot_data = plot_data[(plot_data['induced']=='Induced') & (plot_data['Trip_purpose']=='Work')]

t1 = plot_data.groupby(['user_id','Mode_confirm'], as_index=False).count()[['user_id','Mode_confirm','distance_miles']]
t1['distance_miles'].fillna(0, inplace=True)
t2 = plot_data.groupby(['user_id'], as_index=False).count()[['user_id','distance_miles']]
plot_data = t1.merge(t2, on='user_id')
plot_data['proportion'] = plot_data['distance_miles_x'] / plot_data['distance_miles_y']
plot_data['proportion'].fillna(0, inplace=True)
t3 = data.copy().groupby(['occupation_cat','user_id'], as_index=False).nth(0)[['occupation_cat','user_id']]

plot_data = plot_data[plot_data['Mode_confirm']=='E-bike']
plot_data = plot_data.merge(t3, on='user_id')

plot_title = 'Induced Work E-bike Trips by Occupation Categories'
ylab='Proportion of Total Trips'
fig, ax = plt.subplots(figsize=(8,6))
sns.barplot(data= plot_data, x='proportion' , y='occupation_cat', estimator=np.mean).set(title=plot_title,xlabel=ylab,ylabel='Occupation Category')
plt.xticks(rotation=35, ha='right', fontsize=14)
plt.yticks(fontsize=14)
plt.subplots_adjust(bottom=0.25)

plt.savefig("CanBikeCO_report_occ_induced_ebike_trips.jpeg", bbox_inches='tight')

## substitutions
Figure num 23

In [ ]:
# Substitution rate of ebike trips
plot_data = cleaned_data.copy()
plot_data = plot_data[plot_data['Mode_confirm']=='E-bike']
t1 = plot_data.groupby(['Mode_confirm','Replaced_mode'], as_index=False).count()[['Mode_confirm','Replaced_mode','distance_miles']]
t1['distance_miles'].fillna(0, inplace=True)
t2 = plot_data.groupby(['Mode_confirm'], as_index=False).count()[['Mode_confirm','distance_miles']]
plot_data = t1.merge(t2, on='Mode_confirm')
plot_data['proportion'] = plot_data['distance_miles_x'] / plot_data['distance_miles_y']
plot_data['proportion'].fillna(0, inplace=True)

data_order = plot_data.groupby(['Replaced_mode'], as_index=False).mean().sort_values('proportion', ascending=False).Replaced_mode
labels = plot_data.groupby(['Replaced_mode'], as_index=False).mean().sort_values('proportion', ascending=False).Replaced_mode

plot_title='Stated Replacement for E-Bike Trips'
ylab='Proportion of Trips'

fig, ax = plt.subplots(figsize=(10,4))
sns.barplot(data=plot_data, x='Replaced_mode', y='proportion', estimator=np.mean, order=data_order).set(title=plot_title,xlabel='',ylabel=ylab,ylim=(0,.5))
plt.xticks(rotation=35, ha='right')
plt.subplots_adjust(bottom=0.25)
ax.bar_label(ax.containers[0], fmt='%.2f', padding=10)

## E-bike Mileage change over time
Figure #20

In [ ]:
# How ebike mileage changes over time
from datetime import datetime
plot_data = cleaned_data.copy()
plot_data ['date'] = pd.to_datetime(plot_data['date_time'])
plot_data['Program'] = plot_data['program'].replace(['4c','cc','fc','pc','sc','vail'],['Four Corners (Durango)','Community Cycles (Boulder)','Fort Collins','Pueblo County','Smart Commute (Denver North)','Vail'])
t1 = plot_data.groupby(['user_id','date','Mode_confirm'], as_index=False).sum()[['user_id','date','Mode_confirm','distance_miles']]
t1['distance_miles'].fillna(0, inplace=True)
t2 = plot_data.groupby(['user_id','Program','date'], as_index=False).sum()[['user_id','Program','date','distance_miles']]
plot_data = t1.merge(t2, on=['user_id','date'])
plot_data['proportion'] = plot_data['distance_miles_x'] / plot_data['distance_miles_y']
plot_data['proportion'].fillna(0, inplace=True)
plot_data = plot_data[plot_data['Mode_confirm']=='E-bike']
plot_data = plot_data[plot_data['distance_miles_y'].notnull()]

plot_title = 'E-Bike Mileage Proportion Over Time'
fig, ax = plt.subplots(6,1, figsize=(50,50))
sns.lineplot(data=plot_data[plot_data.Program == "Four Corners (Durango)"], x='date', y='proportion',color="red", hue='Program', estimator=np.mean, ax=ax[0])
ax[0].set_title(plot_title, fontsize=40)
sns.lineplot(data=plot_data[plot_data.Program == "Community Cycles (Boulder)"], x='date', y='proportion', color="blue",hue='Program',estimator=np.mean, ax=ax[1], palette=["C1"])
sns.lineplot(data=plot_data[plot_data.Program == "Fort Collins"], x='date', y='proportion', color="green",hue='Program',estimator=np.mean, ax=ax[2], palette=["C2"])
sns.lineplot(data=plot_data[plot_data.Program == "Pueblo County"], x='date', y='proportion',color="cyan",hue='Program', estimator=np.mean, ax=ax[3], palette=["C3"])
sns.lineplot(data=plot_data[plot_data.Program == "Smart Commute (Denver North)"], x='date', y='proportion',color="purple",hue='Program', estimator=np.mean, ax=ax[4], palette=["C4"])
sns.lineplot(data=plot_data[plot_data.Program == "Vail"], x='date', y='proportion', color="orange",hue='Program',estimator=np.mean, ax=ax[5], palette=["C5"])
ax[5].set(xlabel='Date')
plt.setp(ax[0].get_legend().get_texts(), fontsize='40')
plt.setp(ax[1].get_legend().get_texts(), fontsize='40')
plt.setp(ax[2].get_legend().get_texts(), fontsize='40')
plt.setp(ax[3].get_legend().get_texts(), fontsize='40')
plt.setp(ax[4].get_legend().get_texts(), fontsize='40')
plt.setp(ax[5].get_legend().get_texts(), fontsize='40')
plt.xticks(rotation=35, ha='right', fontsize=20)
plt.subplots_adjust(bottom=0.30)
fig.savefig(r'mileage_over_time', bbox_inches='tight')

## distributions of distances and durations by mode

Figures #4a and 4b

In [ ]:
sns.set_palette('Set1', 9)

In [ ]:
# Distribution of distances
plot_data = cleaned_data.copy()
plot_title = 'Distribution of Trips Durations by Mode'
ylab = 'Duration (minutes)'

fig, ax = plt.subplots(figsize=(10,8))
sns.boxplot(ax=ax, data=plot_data, x='Mode_confirm', y='duration', hue='Mode_confirm', showfliers=False).set(title=plot_title, xlabel='', ylabel=ylab)
plt.subplots_adjust(bottom=0.25)
plt.xticks(rotation=35, ha='right')
plt.legend([])

In [ ]:
# Distribution of distances by program
plot_data = cleaned_data.copy()

plot_title = 'Distribution of Distances by Mode'
ylab = 'Distance (miles)'

fig, ax = plt.subplots(figsize=(10,8))
sns.boxplot(ax=ax, data=plot_data, x='Mode_confirm', y='distance_miles', hue='Mode_confirm', showfliers=False).set(title=plot_title, xlabel='', ylabel=ylab)
plt.subplots_adjust(bottom=0.25)
plt.xticks(rotation=35, ha='right')
plt.legend([])

In [ ]:
sns.set_palette('Set2', 6)

### Proportion of trips that are ebike by income group

Figure #12

In [ ]:
# Proportion of trips that are ebike by income group
plot_data = cleaned_data.copy()

t1 = plot_data.groupby(['user_id','Mode_confirm'], as_index=False).count()[['user_id','Mode_confirm','distance_miles']]
t1['distance_miles'].fillna(0, inplace=True)
t2 = plot_data.groupby(['user_id'], as_index=False).count()[['user_id','distance_miles']]
plot_data = t1.merge(t2, on='user_id')
plot_data['proportion'] = plot_data['distance_miles_x'] / plot_data['distance_miles_y']
plot_data['proportion'].fillna(0, inplace=True)
t3 = data.copy().groupby(['HHINC','user_id'], as_index=False).nth(0)[['HHINC','user_id']]

plot_data = plot_data[plot_data['Mode_confirm']=='E-bike']
plot_data = plot_data.merge(t3, on='user_id')

file_name = 'CanBikeCO_report_income_trips'
ylab='Proportion of Total Trips'
plot_title = 'E-bike Use (Trips) by Income'
fig, ax = plt.subplots(figsize=(6,4))
sns.barplot(data=plot_data, x='HHINC', y='proportion', estimator=np.mean).set(title=plot_title,xlabel='',ylabel=ylab)
plt.xticks(rotation=35, ha='right')
plt.subplots_adjust(bottom=0.25)
ax.figure.savefig(file_name+".png", bbox_inches='tight')

## Proportion of miles that are ebike by income group

figure #15

In [ ]:
# Proportion of miles that are ebike by income group
plot_data = cleaned_data.copy()

t1 = plot_data.groupby(['user_id','Mode_confirm'], as_index=False).sum()[['user_id','Mode_confirm','distance_miles']]
t1['distance_miles'].fillna(0, inplace=True)
t2 = plot_data.groupby(['user_id'], as_index=False).sum()[['user_id','distance_miles']]
plot_data = t1.merge(t2, on='user_id')
plot_data['proportion'] = plot_data['distance_miles_x'] / plot_data['distance_miles_y']
plot_data['proportion'].fillna(0, inplace=True)
t3 = data.copy().groupby(['HHINC','user_id'], as_index=False).nth(0)[['HHINC','user_id']]

plot_data = plot_data[plot_data['Mode_confirm']=='E-bike']
plot_data = plot_data.merge(t3, on='user_id')

file_name = 'CanBikeCO_report_income_mileages'
ylab='Proportion of Total Mileage'
plot_title = 'E-bike Use (Miles) by Income'
fig, ax = plt.subplots(figsize=(6,4))
sns.barplot(data=plot_data, x='HHINC', y='proportion', estimator=np.mean).set(title=plot_title,xlabel='',ylabel=ylab)
plt.xticks(rotation=35, ha='right')
plt.subplots_adjust(bottom=0.25)
ax.figure.savefig(file_name+".png", bbox_inches='tight')

## Proportion of trips that are ebike by income group

figure #14

In [ ]:
sns.set_palette('Paired', 5)

In [ ]:
plot_data = cleaned_data.copy()

t1 = plot_data.groupby(['user_id','Mode_confirm'], as_index=False).count()[['user_id','Mode_confirm','distance_miles']]
t1['distance_miles'].fillna(0, inplace=True)
t2 = plot_data.groupby(['user_id'], as_index=False).count()[['user_id','distance_miles']]
plot_data = t1.merge(t2, on='user_id')
plot_data['proportion'] = plot_data['distance_miles_x'] / plot_data['distance_miles_y']
plot_data['proportion'].fillna(0, inplace=True)
t3 = data.copy().groupby(['VEH_num','user_id'], as_index=False).nth(0)[['VEH_num','user_id']]

plot_data = plot_data[plot_data['Mode_confirm']=='E-bike']
plot_data = plot_data.merge(t3, on='user_id')

file_name = 'CanBikeCO_report_veh_trips'
plot_title = 'E-bike Use (Trips) by Available Vehicles'
ylab='Proportion of Total Trips'
fig, ax = plt.subplots(figsize=(6,4))
sns.barplot(data=plot_data, x='VEH_num', y='proportion', estimator=np.mean).set(title=plot_title,xlabel='',ylabel=ylab)
plt.xticks(rotation=35, ha='right')
plt.subplots_adjust(bottom=0.25)
ax.figure.savefig(file_name+".png", bbox_inches='tight')

## mileage over time

figure #18 and #19

In [ ]:
sns.set_palette('Set1', 3)

In [ ]:
# How total mileage changes over time
plot_data = cleaned_data.copy()
plot_data ['date_time'] = pd.to_datetime(plot_data['date_time'])

#need to count all combinations - make sure to treat as categorical
plot_data["User"] = plot_data["user_id"].astype("category")
plot_data["Date"] = plot_data["date_time"].astype("category")
plot_data = plot_data.groupby(['User', 'Date']).distance_miles.sum().reset_index()

plot_data

plot_title = 'Total Mileage Over Time'
ylab = 'Daily Miles per User'
file_name = "CanBikeCO_report_ts_miles"
fig, ax = plt.subplots(figsize=(16,4))
sns.lineplot(data=plot_data, x='Date', y='distance_miles', estimator=np.mean).set(title=plot_title, xlabel='Date', ylabel=ylab)
plt.xticks(rotation=35, ha='right')
plt.subplots_adjust(bottom=0.25)
ax.figure.savefig(file_name+".png", bbox_inches='tight')

In [ ]:
# How ebike mileage changes over time
plot_data = cleaned_data.copy()
plot_data['date_time'] = pd.to_datetime(plot_data['date_time'])

#treat as categorical to count all combinations
plot_data["user_id"] = plot_data["user_id"].astype("category")
plot_data["date_time"] = plot_data["date_time"].astype("category")
plot_data['Mode_confirm'] = plot_data['Mode_confirm'].astype("category")

#using the total mileage data as one side
t1 = plot_data.groupby(['user_id', 'date_time']).distance_miles.sum().reset_index()

#create the other side -- also grouping by mode
t2 = plot_data.groupby(['user_id', 'date_time', 'Mode_confirm']).distance_miles.sum().reset_index()

t2.sample(n=50, random_state=321)

#then we merge
plot_data = t2.merge(t1, on=['user_id','date_time'])
plot_data = plot_data[plot_data['Mode_confirm']=='E-bike']
plot_data.sample(n=50, random_state=321)

plot_data['proportion'] = plot_data['distance_miles_x'] / plot_data['distance_miles_y']
plot_data['proportion'].fillna(0, inplace=True)

plot_data.sample(n=50, random_state=321)

plot_data = plot_data[plot_data['distance_miles_y'].notnull()]
plot_data = plot_data[plot_data['distance_miles_y'] != 0] #drop 0 mile days to prevent 0/0 reading as 1

plot_title = 'E-Bike Mileage Proportion Over Time'
ylab = 'Proportion of Daily Miles'
file_name = "CanBikeCO_report_ts_mileage_proportion"
fig, ax = plt.subplots(figsize=(16,4))
sns.lineplot(data=plot_data, x='date_time', y='proportion', estimator=np.mean).set(title=plot_title, xlabel='Date', ylabel=ylab)
plt.xticks(rotation=35, ha='right')
plt.subplots_adjust(bottom=0.25)
ax.figure.savefig(file_name+".png", bbox_inches='tight')

## Distance and mode!

figure #16 & 17

In [ ]:
sns.set_palette('Set1', 9)

In [ ]:
# Distance and mode chosen relationship -- short trips
plot_data = cleaned_data.copy()
plot_data = plot_data[plot_data['distance_miles']<6]

plot_title = 'Mode Share by Trip Distance'
ylab = 'Total Trips'
file_name = "CanBikeCO_report_mode_share_distance_short"
sns.histplot(plot_data, x="distance_miles", hue="Mode_confirm", element="poly", multiple="stack")
# proportion_hist_plot(plot_data, 'distance_miles', plot_title, ylab, file_name)

In [ ]:
# Distance and mode chosen relationship -- long trips
plot_data = cleaned_data.copy()
plot_data = plot_data[plot_data['distance_miles']>5]

plot_title = 'Mode Share by Trip Distance'
ylab = 'Total Trips'
file_name = "CanBikeCO_report_mode_share_distance_long"
# proportion_hist_plot(plot_data, 'distance_miles', plot_title, ylab, file_name)
sns.histplot(plot_data, x="distance_miles", hue="Mode_confirm", element="poly", multiple="stack")

## induced trip purposes for e-bikes

Figure #24

In [ ]:
cleaned_data.head()

In [ ]:
# What purpose ebike is used for by program
plot_data = cleaned_data.copy()
t1 = plot_data.groupby(['induced','Trip_purpose','Mode_confirm'], as_index=False).count()[['induced','Trip_purpose','Mode_confirm','distance_miles']]
t1['distance_miles'].fillna(0, inplace=True)
t2 = plot_data.groupby(['induced','Trip_purpose'], as_index=False).count()[['induced','Trip_purpose','distance_miles']]
plot_data = t1.merge(t2, on=['induced','Trip_purpose'])
plot_data['proportion'] = (plot_data['distance_miles_x'] / plot_data['distance_miles_y'])*100
plot_data['proportion'].fillna(0, inplace=True)
plot_data = plot_data[plot_data['Mode_confirm']=='E-bike']
plot_data = plot_data[plot_data['induced']=='Induced']

plot_title = 'Induced E-Bike Trip Proportion By Trip Purpose'
ylab = 'Proportion of Induced Trips'
fig, ax = plt.subplots(figsize=(20,10))
sns.barplot(data=plot_data, x='Trip_purpose', y='proportion', hue='induced',estimator=np.mean, ci=None).set(title=plot_title, xlabel='Date', ylabel=ylab)
plt.xticks(rotation=35, ha='right')
plt.subplots_adjust(bottom=0.25)
